In [ ]:
# @title downloads e importações {"display-mode":"form"}

! pip install imblearn gdown keras_tuner keras

import warnings # type: ignore
warnings.filterwarnings("ignore")

import pandas as pd # type: ignore
import numpy as np # type: ignore
from sklearn.model_selection import train_test_split # type: ignore
from sklearn.preprocessing import RobustScaler # type: ignore
from imblearn.over_sampling import SMOTE # type: ignore
import seaborn as sns # type: ignore
import matplotlib.pyplot as plt # type: ignore
import gdown # type: ignore
from sklearn.cluster import KMeans # type: ignore
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay # type: ignore
import keras_tuner as kt # type: ignore
import tensorflow.keras as keras # type: ignore
from tensorflow.keras import layers # type: ignore
from mpl_toolkits.mplot3d import Axes3D # type: ignore


# ids = {
#     "consumo_2024": "1-iXT7eaJWQokHf9cyfrB8N5wvkdhgjJW",
#     "consumo_2023": "1-WfvkRwaRr85B_Joxcm9xVdpyg5NBAmp",
#     "consumo_2022": "1-Uu4Tf4lufJVFeJnYKc5w7OeW66pe1eC",
#     "consumo_2021": "1-2PsTLzG4dcY4wM0p7vFfabUuLv950gC",
#     "consumo_2020": "1-1pOoa0eJlNJ94BMi7p4PTx5KUS96mhX",
#     "consumo_2019": "1-2PsTLzG4dcY4wM0p7vFfabUuLv950gC",
#     "fraudes": "1-MbIlChqQapcxFkoJgpbQIsN9FBLfbX1"
# }

ids = {
    "consumo_2023": "1-WfvkRwaRr85B_Joxcm9xVdpyg5NBAmp",
    "fraudes": "1-MbIlChqQapcxFkoJgpbQIsN9FBLfbX1"
}

for key, file_id in ids.items():
    url = f"https://drive.google.com/uc?id={file_id}"
    arquivo_destino = "dataset_{}.csv".format(key)
    gdown.download(url, arquivo_destino, quiet=False)

In [ ]:
# @title utils {"display-mode":"form"}

# funcs

def reduzir_dataset(df, n=10000):
    """ diminui o dataset, levando em consideração a matrícula
    """
    matriculas = df['MATRICULA'].unique()[:n]
    df_filtrado = df[df['MATRICULA'].isin(matriculas)]
    return df_filtrado


# plot

def plotar_clusters(df, n_clusters):
    """
    plota os clusters em um gráfico de dispersão.
    """
    plt.figure(figsize=(10, 6))
    plt.scatter(df['COD_LONGITUDE'], df['COD_LATITUDE'], c=df['cluster'], cmap='viridis', marker='o', s=100)
    plt.title(f"K-Means Clustering com {n_clusters} Clusters")
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.show()

def plotar_box_plot(df_filtrado, column_name, print_text):
    """
    plota um boxplot para uma coluna específica do DataFrame.
    """
    sns.boxplot(x=df_filtrado[column_name])
    plt.title(f'Boxplot de {column_name} ({print_text})')
    plt.xlabel(column_name)
    plt.show()

def plotar_matrix_de_consusao(cm):
    """
    plota uma matriz de confusão para avaliar o desempenho do modelo.
    """
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap=plt.cm.Blues)
    plt.title('Matriz de Confusão - Melhor Modelo')
    plt.xlabel('Predito')
    plt.ylabel('Verdadeiro')
    plt.show()

def plotar_hiperparametros_2d():
    """
    plota hiperparametros 2d
    """
    trial_data = []

    for trial in tuner.oracle.get_best_trials(num_trials=20):
        data = trial.hyperparameters.values.copy()
        data['score'] = trial.score

        units_keys = [key for key in data.keys() if 'units_' in key]
        dropout_keys = [key for key in data.keys() if 'dropout_' in key]

        data['units_mean'] = np.mean([data[key] for key in units_keys])
        data['dropout_mean'] = np.mean([data[key] for key in dropout_keys])

        trial_data.append(data)

    df = pd.DataFrame(trial_data)

    hp_x = 'learning_rate'
    hp_y = 'units_0'

    plt.figure(figsize=(10, 8))
    sns.scatterplot(data=df, x=hp_x, y=hp_y, size='score', hue='score', palette='viridis', legend='full')
    plt.title(f'Interação entre {hp_x} e {hp_y}')
    plt.xlabel(hp_x)
    plt.ylabel(hp_y)
    plt.show()

def plotar_hiperparametros_3d(tuner):
    """
    plota hiperparâmetros em um gráfico 3D para análise de resultados.
    """
    trial_data = []

    for trial in tuner.oracle.get_best_trials(num_trials=20):
        data = trial.hyperparameters.values.copy()
        data['score'] = trial.score

        units_keys = [key for key in data.keys() if 'units_' in key]
        dropout_keys = [key for key in data.keys() if 'dropout_' in key]

        data['units_mean'] = np.mean([data[key] for key in units_keys])
        data['dropout_mean'] = np.mean([data[key] for key in dropout_keys])

        trial_data.append(data)

    df = pd.DataFrame(trial_data)

    hp_x = 'learning_rate'
    hp_y = 'units_mean'
    hp_z = 'dropout_mean'

    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    scat = ax.scatter(df[hp_x], df[hp_y], df[hp_z], c=df['score'], cmap='viridis', s=50)
    ax.set_xlabel('Learning Rate')
    ax.set_ylabel('Média de Units')
    ax.set_zlabel('Média de Dropout')
    ax.set_title('Visualização 3D dos Hiperparâmetros (Médias)')
    fig.colorbar(scat, label='Score de Validação (acurácia)')
    plt.show()

def plot_hiperparametros_heatmap():
  """
  plota um heatmap com os hiperparâmetros e a acurácia
  """
  trial_data = []

  for trial in tuner.oracle.get_best_trials(num_trials=20):
        data = trial.hyperparameters.values.copy()
        data['score'] = trial.score

        units_keys = [key for key in data.keys() if 'units_' in key]
        dropout_keys = [key for key in data.keys() if 'dropout_' in key]

        data['units_mean'] = np.mean([data[key] for key in units_keys])
        data['dropout_mean'] = np.mean([data[key] for key in dropout_keys])

        trial_data.append(data)

  df = pd.DataFrame(trial_data)

  hp1 = 'units_mean'
  hp2 = 'dropout_mean'

  # Se os hiperparâmetros não existirem, calcule as médias como anteriormente
  if 'units_mean' not in df.columns:
      units_keys = [key for key in df.columns if 'units_' in key]
      df['units_mean'] = df[units_keys].mean(axis=1)

  if 'dropout_mean' not in df.columns:
      dropout_keys = [key for key in df.columns if 'dropout_' in key]
      df['dropout_mean'] = df[dropout_keys].mean(axis=1)

  # Criar tabela dinâmica
  pivot_table = df.pivot_table(values='score', index=hp1, columns=hp2)

  # Plotar heatmap
  plt.figure(figsize=(10, 8))
  sns.heatmap(pivot_table, annot=True, fmt=".4f", cmap='viridis')
  plt.title(f'Heatmap de {hp1} vs {hp2}')
  plt.xlabel(hp2)
  plt.ylabel(hp1)
  plt.show()


# Modelagem dos dados

In [ ]:
# adicione caso quera todos os anos
# [
#     "./dataset_consumo_2024.csv",
#     "./dataset_consumo_2023.csv",
#     "./dataset_consumo_2022.csv",
#     "./dataset_consumo_2021.csv",
#     "./dataset_consumo_2019.csv"
# ]


arquivos_csv = [
    "./dataset_consumo_2023.csv"
]

all_columns_geral = pd.concat([pd.read_csv(arquivo, delimiter=";") for arquivo in arquivos_csv], axis=0)
fraudes = pd.read_csv('./dataset_fraudes.csv')

In [ ]:
# troque para True caso queria diminuir do dataset para ficar mais maleável

if False: df = reduzir_dataset(all_columns_geral)

In [ ]:
# filtrando pela de 2023

fraudes = fraudes[fraudes["ANOOS"] == 2023]
fraudes = fraudes.rename(columns={'DESCRICAO_IRREGULARIDADE IDENTIFICADA': 'IRREGULARIDADE'})

In [ ]:
# limitando apenas para a categoria residencial

all_columns_geral = all_columns_geral[all_columns_geral["CATEGORIA"].isin(["RESIDENCIAL"])]
all_columns_geral = all_columns_geral.drop(columns="CATEGORIA")

In [ ]:
# removendo colunas que não seram utilizandas

all_columns_geral = all_columns_geral.drop(columns=['Unnamed: 0',"FATURADO_MEDIA", "SUB_CATEGORIA", "STA_TROCA", "STA_ACEITA_LEITURA", 'STA_TROCA', 'EMP_CODIGO', 'COD_GRUPO', 'COD_SETOR_COMERCIAL', 'NUM_QUADRA', 'COD_ROTA_LEITURA', 'SEQ_RESPONSAVEL', 'ECO_RESIDENCIAL', 'ECO_COMERCIAL', 'ECO_INDUSTRIAL', 'ECO_PUBLICA', 'ECO_OUTRAS','LTR_ATUAL', 'LTR_COLETADA', 'DAT_LEITURA', 'DIAS_LEITURA', 'COD_LEITURA_INF_1', 'COD_LEITURA_INF_2', 'COD_LEITURA_INF_3', 'HORA_LEITURA', 'DSC_SIMULTANEA', 'COD_LEITURA_INT','EXCECAO'])

In [ ]:
# apagando os valores nulos

sem_na = all_columns_geral.dropna()

print(f"quantidade de linhas antes de remover nulos: {len(all_columns_geral)}")
print(f"quantidade de linhas após de remover nulos: {len(sem_na)}")

In [ ]:
# remove outliers de latitude e longetude

def remove_outliers_lat_long(df, column_names):
    condition = (df[column_names] > -15).any(axis=1)
    df_filtered = df[~condition]
    return df_filtered

df_sem_linhas = remove_outliers_lat_long(sem_na, ['COD_LATITUDE', 'COD_LONGITUDE'])

In [ ]:
# clusterizando os dados de longetude e latitude

kmeans = KMeans(n_clusters=40, random_state=42)
df_sem_linhas['cluster'] = kmeans.fit_predict(df_sem_linhas[['COD_LATITUDE', 'COD_LONGITUDE']])

# plotando os clusters
plotar_clusters(df_sem_linhas, kmeans.n_clusters)

# removendo colunas de longetude e latude
df_loc_tratado = df_sem_linhas.drop(columns=['COD_LATITUDE', 'COD_LONGITUDE'])

In [ ]:
# feature de estimativa de consumo por cluster

# TODO:.

In [ ]:
# fazendo one hot encoding

df_loc_tratado_ohc = pd.get_dummies(df_loc_tratado, columns = ['cluster'], dtype=int)

In [ ]:
df_antes_de_tratar = df_loc_tratado_ohc.copy()
df_loc_tratado_ohc["TIPO_LIGACAO"] = df_loc_tratado_ohc["TIPO_LIGACAO"].replace({"Hidrometrado": 0, "Consumo Fixo": 1})

print(f"antes de tratar: {df_antes_de_tratar['TIPO_LIGACAO'].dtypes}"); del df_antes_de_tratar
print(f"depois de tratar: {df_loc_tratado_ohc['TIPO_LIGACAO'].dtypes}")

In [ ]:
# fazendo a normalização

def normalizar(df):
    scaler = RobustScaler()
    df[['VOLUME_ESTIMADO_ACUM', 'VOLUME_ESTIMADO']] = scaler.fit_transform(df[['VOLUME_ESTIMADO_ACUM', 'VOLUME_ESTIMADO']])
    return df

print(f"antes de normalizar:\n{df_loc_tratado_ohc[['VOLUME_ESTIMADO_ACUM', 'VOLUME_ESTIMADO']].median()}\n")
df_loc_tratado_ohc = normalizar(df_loc_tratado_ohc)
print(f"depois de normalizar:\n{df_loc_tratado_ohc[['VOLUME_ESTIMADO_ACUM', 'VOLUME_ESTIMADO']].median()}")

In [ ]:
# pivotando a tabela

def pivot_table_by_reference(df):
    pivoted_df = pd.pivot_table(
        df,
        index='MATRICULA',
        columns='REFERENCIA',
        values=['CONS_MEDIDO', 'VOLUME_ESTIMADO'],
        aggfunc='sum'
    )

    pivoted_df.columns = ['_'.join(col).strip() for col in pivoted_df.columns.values]
    pivoted_df = pivoted_df.reset_index()

    other_columns = df.drop(columns=['CONS_MEDIDO', 'VOLUME_ESTIMADO', 'REFERENCIA']).drop_duplicates()
    pivoted_df = pd.merge(pivoted_df, other_columns, on='MATRICULA', how='left')
    return pivoted_df.fillna(0)


print(f"antes de pivotar: \n{df_loc_tratado_ohc.columns}\n")
pivoted_df = pivot_table_by_reference(df_loc_tratado_ohc)
print(f"depois de pivotar: \n{pivoted_df.columns}")

In [ ]:
# balanciando e simplificando a coluna do tipo de ocorrencia para normal e não normal

result = pivoted_df.copy()
result['DSC_OCORRENCIA'] = pivoted_df['DSC_OCORRENCIA'].apply(lambda x: 1 if x == 'NORMAL' else 0)

print(f"antes da transformação: {pivoted_df['DSC_OCORRENCIA'].value_counts()}")
print(f"\napós da transformação: {result['DSC_OCORRENCIA'].value_counts()}")

In [ ]:
# removendo outliers

def remover_outliers(df, column_name):
    # calculando os quartis
    Q1 = df[column_name].quantile(0.25)
    Q3 = df[column_name].quantile(0.75)

    # calculando o IQR
    IQR = Q3 - Q1

    # 3. Determinar os limites superior e inferior
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR

    # filtrar os dados para remover os outliers
    df_filtrado = df[(df[column_name] >= limite_inferior) & (df[column_name] <= limite_superior)]

    return df_filtrado

df_sem_outliers = remover_outliers(result, 'VOLUME_ESTIMADO_ACUM')

In [ ]:
# tratando a tabela de fraudes

dataframe_fraudes_premissa = fraudes[['MATRICULA', 'DESCRICAO']].drop_duplicates()
dataframe_fraudes_premissa = pd.get_dummies(dataframe_fraudes_premissa, columns=['DESCRICAO'], dtype='int')

In [ ]:
# juntando a tabela de fraudes com a de consumo

df_sem_outliers["MATRICULA"] = df_sem_outliers["MATRICULA"].astype(int)

dataframe_pf_premissa = pd.merge(df_sem_outliers, dataframe_fraudes_premissa, on='MATRICULA', how='left')

dataframe_pf_premissa = dataframe_pf_premissa.rename(columns={"DESCRICAO_IRREGULARIDADE IDENTIFICADA": "IRREGULARIDADE"})

dataframe_pf_premissa['IRREGULARIDADE'] = dataframe_pf_premissa['IRREGULARIDADE'].fillna(0).astype(int)

In [ ]:
dataframe_pf_premissa

In [ ]:
# balanciamento os dados

def balanciar(df):
  smote = SMOTE(random_state=42)
  X = df.drop('IRREGULARIDADE', axis=1)
  y = df['IRREGULARIDADE']
  X_res, y_res = smote.fit_resample(X, y)
  return pd.concat([X_res, y_res], axis=1)

verificar_quantidade = lambda df: np.unique(df['IRREGULARIDADE'].values, return_counts=True)

print(f"antes de balanciar: {dataframe_pf_premissa['IRREGULARIDADE'].unique}")
dataframe_pf_premissa = balanciar(dataframe_pf_premissa)
print(f"depois de balanciar: {dataframe_pf_premissa['IRREGULARIDADE'].unique}")

# Modelo

In [ ]:
# dividindo as features das targuets

y = dataframe_pf_premissa['IRREGULARIDADE'].values
X = dataframe_pf_premissa.drop(['MATRICULA','IRREGULARIDADE'], axis=1).values

In [ ]:
# fazendo a divisão de treino e teste

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [ ]:
# Definindo a função para construir o modelo
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Input(shape=(X_train.shape[1],)))

    # Hyperparameters para o número de unidades nas camadas
    for i in range(hp.Int('num_layers', 2, 4)):  # entre 2 e 4 camadas densas
        model.add(keras.layers.Dense(units=hp.Int(f'units_{i}', min_value=64, max_value=512, step=64), activation='relu'))
        model.add(keras.layers.Dropout(rate=hp.Float(f'dropout_{i}', min_value=0.2, max_value=0.5, step=0.1)))

    model.add(keras.layers.Dense(1, activation='sigmoid'))

    # Hyperparameters para o otimizador e a taxa de aprendizado
    model.compile(
        optimizer=keras.optimizers.Adam(
            learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')),
        loss='binary_crossentropy',
        metrics=['accuracy', 'precision'])

    return model

# Definindo o tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=2,
    directory='my_dir',
    project_name='keras_tuning')

# Iniciando o grid search
tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

In [ ]:
for i, model in enumerate(best_models):
    print(f"\nArquitetura do Modelo n{i+1}:")
    model.summary()

In [ ]:
# fazer teste com o melhor modelo para criar métricas

best_model = tuner.get_best_models(num_models=1)[0]

y_pred_prob = best_model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype("int32")

In [ ]:
# matriz de confusão

cm = confusion_matrix(y_test, y_pred)
plotar_matrix_de_consusao(cm)

Nessa matriz de confusão, podemos ver bons resultados, com uma quantidade relativamente baixa de falsos negativos (25.992) e falsos positivos (8.328). Nossa principal métrica, sendo a precisão, reflete o quão bem o modelo conseguiu classificar corretamente as instâncias positivas. Com um número elevado de verdadeiros positivos (75.376) e poucos falsos positivos, podemos dizer que a precisão do modelo é boa. Dessa forma, podemos inferir que o modelo encontrado através do random search foi eficaz e, no final, conseguimos obter um bom desempenho preditivo.

In [ ]:
plotar_hiperparametros_2d()

Nesse gráfico 2D, onde podemos observar a correlação entre os resultados dos modelos com base no learning rate e no tamanho do modelo, podemos inferir que os modelos com melhor desempenho foram aqueles que utilizaram uma taxa de aprendizado menor. Além disso, os modelos com tamanhos variando de 250 a 450 apresentaram as melhores performances, sugerindo que essa combinação de parâmetros foi mais eficiente em ajustar o modelo e alcançar bons resultados.

In [ ]:
plotar_hiperparametros_3d(tuner)

Neste gráfico 3D, podemos visualizar a correlação entre o tamanho do modelo, a taxa de aprendizado e a quantidade de dropout, formando uma análise em três dimensões. Podemos inferir que os modelos que performaram melhor foram aqueles com uma taxa de aprendizado menor, com tamanhos moderados, e uma taxa de dropout variando entre 0,2 e 0,38. Essa combinação parece ter contribuído para uma melhor generalização dos modelos.

In [ ]:
plot_hiperparametros_heatmap()

Nesse heatmap, podemos ver a correlação entre o tamanho do modelo e a média de dropout, além da média do tamanho do modelo. Podemos observar que os modelos que performaram melhor foram aqueles com uma média de tamanho de 277 ou menos. Esses modelos, combinados com uma média de dropout adequada, apresentaram os melhores resultados, sugerindo que tamanhos menores e valores de dropout moderados levaram a um melhor desempenho.